# Stop Data
This file will read in the csv data from a file, 
clean and prepare it 
enter it in the stops database

In [10]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime

# Read data from csv file 

In [11]:
stop_file = "static/data/Police_Stop_data.csv"
stop_df=pd.read_csv(stop_file)
stop_df.head(1)

,OBJECTID,masterIncidentNumber,responseDate,reason,problem,callDisposition,citationIssued,personSearch,vehicleSearch,preRace,race,gender,lat,long,x,y,policePrecinct,neighborhood,lastUpdateDate
0,1001,16-406479,2016-11-09T17:43:21.000Z,NaN,Attempt Pick-Up (P),BKG-Booking,NaN,YES,NO,Native American,Native American,Male,44.949552,-93.281346,-1.038403e+07,5.613583e+06,5.0,Whittier,2017-08-08T10:24:35.000Z


# Imports for database


In [12]:
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Set up tables

In [13]:
# Define our neighborhoodData table
class neighborhoodData(Base):
    __tablename__ = 'neighborhoodData'
    id = Column(Integer, primary_key=True)
    neighborhood = Column(String)

In [14]:
# Define our stopData table
class stopData(Base):
    __tablename__ = 'stopData'
    OBJECTID = Column(Integer, primary_key=True)
    neighborhood = Column(String)
    responseDate = Column(String)
    citationIssued = Column(String)
    lat= Column(Integer)
    lon= Column(Integer)
    gender =Column(String)
    responseDow = Column(Integer)
    responseDay =Column(Integer)
    responseMonth = Column(Integer)
    responseMonthName =Column(String)
    responseYear = Column(Integer)
    

In [15]:
# Define dowData table
class dowData(Base):
    __tablename__ = 'dowData'
    neighborhood = Column((String), primary_key=True)
    gender =Column(String)
    responseDow = Column(Integer)
    genderCount = Column(Integer)

In [16]:
# Define our citationData table
class citationData(Base):
    __tablename__ = 'citationData'
    neighborhood = Column(String, primary_key=True)
    citationIssued = Column(String)
    responseDay =Column(Integer)
    responseMonth = Column(Integer)
    responseMonthName =Column(String)
    responseYear = Column(Integer)
    citationCnt = Column(Integer)
    

In [17]:
# tables only in python
Base.metadata.tables
# Create our database engine
engine = create_engine('sqlite:///stops.sqlite')
# This is where we create our tables in the database
Base.metadata.create_all(engine)
# The ORM’s “handle” to the database is the Session.
from sqlalchemy.orm import Session
session = Session(engine)

# Reduce Columns
Reduce the columns to those needed and eliminate rows with null gender  
check count to confirm later operations  

In [18]:
      #select needed columns
select_df=stop_df.loc[:,["OBJECTID", "responseDate","citationIssued","lat","lon","gender","neighborhood"]]

# remove null gender rows
reducedA_df=select_df.dropna(axis=0,subset=['gender'])

# remove null neighborhood rows
reduced_df=reducedA_df.dropna(axis=0,subset=['neighborhood'])
reduced_df.head()

C:\Users\blizz\Anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,OBJECTID,responseDate,citationIssued,lat,lon,gender,neighborhood
0,1001,2016-11-09T17:43:21.000Z,NaN,44.949552,NaN,Male,Whittier
1,1002,2016-11-09T17:58:28.000Z,NaN,45.014157,NaN,Male,Cleveland
4,1005,2016-11-09T18:40:01.000Z,NaN,44.911719,NaN,Unknown,Minnehaha
5,1006,2016-11-09T18:40:44.000Z,NaN,44.976087,NaN,Male,Downtown West
6,1007,2016-11-09T18:51:47.000Z,NaN,44.959130,NaN,Male,Lowry Hill East


# Get a list of neighborhoods
remove 1 null neighborhood

In [19]:
#Get a listing and count of neighborhoods
neighbor_df=reduced_df["neighborhood"]
neighborhood_df=pd.DataFrame(neighbor_df.drop_duplicates())

neighborhood_df.tail()

,neighborhood
1270,Cooper
1317,Bryn - Mawr
2595,University of Minnesota
4425,Field
5897,Humboldt Industrial Area


In [20]:
neighborhood_df.to_csv("static/data/neighborhood_data.csv", index=False,header=True)

# Put neighborhood data in the table

In [21]:
neighborhood_df.to_sql(neighborhoodData.__tablename__, engine, if_exists='replace')

session.commit()

In [22]:
engine.execute('select * from neighborhoodData').fetchall()

[(0, 'Whittier'),
 (1, 'Cleveland'),
 (4, 'Minnehaha'),
 (5, 'Downtown West'),
 (6, 'Lowry Hill East'),
 (7, 'Seward'),
 (9, 'Near - North'),
 (10, 'Marcy Holmes'),
 (11, 'Hawthorne'),
 (17, 'Diamond Lake'),
 (18, 'Midtown Phillips'),
 (21, 'Folwell'),
 (25, 'Willard - Hay'),
 (27, 'Howe'),
 (28, 'Phillips West'),
 (31, 'King Field'),
 (32, 'Ventura Village'),
 (34, 'Jordan'),
 (37, 'Victory'),
 (43, 'Loring Park'),
 (45, 'Lyndale'),
 (50, 'ECCO'),
 (51, 'Bottineau'),
 (54, 'Lind - Bohanon'),
 (56, 'Cedar Riverside'),
 (59, 'Como'),
 (60, 'CARAG'),
 (61, 'Windom'),
 (63, 'North Loop'),
 (64, 'St. Anthony East'),
 (65, 'Standish'),
 (77, 'Webber - Camden'),
 (79, 'Waite Park'),
 (80, 'St. Anthony West'),
 (81, 'Audubon Park'),
 (87, 'Windom Park'),
 (88, "Steven's Square - Loring Heights"),
 (101, 'Central'),
 (102, 'Longfellow'),
 (103, 'Lynnhurst'),
 (105, 'Beltrami'),
 (109, 'Elliot Park'),
 (110, 'Harrison'),
 (111, 'East Phillips'),
 (117, 'Morris Park'),
 (118, 'Nicollet Island - 

# modify Gender 
combine gender Unknown and Gender Non-Conforming into gender Other

In [23]:
pd.options.mode.chained_assignment = None  # default='warn'
#fill in Unknown vaue with Other
reduced_df.loc[reduced_df.gender=="Unknown","gender"] = "Other"
      # fill in Gender Non-Conforming values with other
reduced_df.loc[reduced_df.gender =="Gender Non-Conforming","gender"] = "Other"
reduced_df.head()

,OBJECTID,responseDate,citationIssued,lat,lon,gender,neighborhood
0,1001,2016-11-09T17:43:21.000Z,NaN,44.949552,NaN,Male,Whittier
1,1002,2016-11-09T17:58:28.000Z,NaN,45.014157,NaN,Male,Cleveland
4,1005,2016-11-09T18:40:01.000Z,NaN,44.911719,NaN,Other,Minnehaha
5,1006,2016-11-09T18:40:44.000Z,NaN,44.976087,NaN,Male,Downtown West
6,1007,2016-11-09T18:51:47.000Z,NaN,44.959130,NaN,Male,Lowry Hill East


In [24]:
#what gender values are present ?
#verify count
gender=reduced_df.groupby(["gender"]).count()
gender

,OBJECTID,responseDate,citationIssued,lat,lon,neighborhood
gender,,,,,,
Female,19717,19717,12523,19717,0,19717
Male,55334,55334,36284,55334,0,55334
Other,13398,13398,8912,13398,0,13398


# Modify citationIssued 
change NaN to NO

In [25]:
# change null citationIssued to NO
reduced_df[["citationIssued"]]=reduced_df[["citationIssued"]].fillna(value='NO')
reduced_df.head()

,OBJECTID,responseDate,citationIssued,lat,lon,gender,neighborhood
0,1001,2016-11-09T17:43:21.000Z,NO,44.949552,NaN,Male,Whittier
1,1002,2016-11-09T17:58:28.000Z,NO,45.014157,NaN,Male,Cleveland
4,1005,2016-11-09T18:40:01.000Z,NO,44.911719,NaN,Other,Minnehaha
5,1006,2016-11-09T18:40:44.000Z,NO,44.976087,NaN,Male,Downtown West
6,1007,2016-11-09T18:51:47.000Z,NO,44.959130,NaN,Male,Lowry Hill East


In [26]:
#what gender values are present ?
# verify count
citation=reduced_df.groupby(["citationIssued"]).count()
citation

,OBJECTID,responseDate,lat,lon,gender,neighborhood
citationIssued,,,,,,
NO,79183,79183,79183,0,79183,79183
YES,9266,9266,9266,0,9266,9266


# Parse date
responseDow = day of the week  
responseDay = day of the month as zero padded decimal  
responseDonth = month as a zero padded decimal  
responseMonthName= month as abbreviated name  
responseYear  

In [27]:
#parse date
date_list= [datetime.strptime( entry,"%Y-%m-%dT%H:%M:%S.000Z") for entry in reduced_df["responseDate"]]
# determine response day of week
dow_list= [datetime.strftime(entry,"%w") for entry in date_list]

# split off response day, month and year
day_list = [datetime.strftime(entry,"%d") for entry in date_list]
month_list=[datetime.strftime(entry,"%m") for entry in date_list]
month_name_list=[datetime.strftime(entry,"%b") for entry in date_list]
year_list= [datetime.strftime(entry,"%Y") for entry in date_list]

#add column to reduced_df
reduced_df.loc[:,"responseDow"]=dow_list
reduced_df.loc[:,"responseDay"]=day_list
reduced_df.loc[:,"responseMonth"]=month_list
reduced_df.loc[:,"responseMonthName"]=month_name_list
reduced_df.loc[:,"responseYear"]=year_list
reduced_df.head()

,OBJECTID,responseDate,citationIssued,lat,lon,gender,neighborhood,responseDow,responseDay,responseMonth,responseMonthName,responseYear
0,1001,2016-11-09T17:43:21.000Z,NO,44.949552,NaN,Male,Whittier,3,09,11,Nov,2016
1,1002,2016-11-09T17:58:28.000Z,NO,45.014157,NaN,Male,Cleveland,3,09,11,Nov,2016
4,1005,2016-11-09T18:40:01.000Z,NO,44.911719,NaN,Other,Minnehaha,3,09,11,Nov,2016
5,1006,2016-11-09T18:40:44.000Z,NO,44.976087,NaN,Male,Downtown West,3,09,11,Nov,2016
6,1007,2016-11-09T18:51:47.000Z,NO,44.959130,NaN,Male,Lowry Hill East,3,09,11,Nov,2016


# Deliver clean stop data
to file "static/data/clean_data.csv"  
or to database

In [28]:
reduced_df.to_csv("static/data/clean_stop_data.csv", index=False,header=True)

# Add Stop data to the database

In [29]:
reduced_df.to_sql(stopData.__tablename__, engine, if_exists='replace',chunksize=50)

session.commit()

In [30]:
#test for data
neighbor = session.query(stopData.neighborhood, stopData.gender).filter_by(neighborhood="Como").first()
neighbor

('Como', 'Male')

# Prep responseDow data
get count by neighborhood, responseDow and gender

In [31]:
# get count of genders by neighborhood and responseDow
group_df=reduced_df.groupby(["neighborhood","responseDow","gender"]).count()
group_df.head()

OBJECTID  responseDate  citationIssued  lat  \
neighborhood responseDow gender                                                
Armatage     0           Female         1             1               1    1   
                         Male           9             9               9    9   
                         Other          9             9               9    9   
             1           Female         2             2               2    2   
                         Male           6             6               6    6   

                                 lon  responseDay  responseMonth  \
neighborhood responseDow gender                                    
Armatage     0           Female    0            1              1   
                         Male      0            9              9   
                         Other     0            9              9   
             1           Female    0            2              2   
                         Male      0            6              6   

                                 responseMonthName  responseYear  
neighborhood responseDow gender                                   
Armatage     0           Female                  1             1  
                         Male                    9             9  
                         Other                   9             9  
             1           Female                  2             2  
                         Male                    6             6

In [32]:
# remove indexing rename columns and reduce columns
new_group=group_df.reset_index()
next_group=new_group.rename(columns={"OBJECTID":"genderCount"})
dow_group=next_group[["neighborhood","responseDow","gender","genderCount"]]
dow_group.head()

,neighborhood,responseDow,gender,genderCount
0,Armatage,0,Female,1
1,Armatage,0,Male,9
2,Armatage,0,Other,9
3,Armatage,1,Female,2
4,Armatage,1,Male,6


# Deliver clean day of week data  
to file "static/data/dow_clean_data.csv" 
or database

In [ ]:
# write ad csv file -- json file doesn't come out right
dow_group.to_csv("static/data/clean_dow_data.csv" ,index=False, header=True)

# Add dow Data to database

In [ ]:
dow_group.to_sql(dowData.__tablename__, engine, if_exists='replace',chunksize=50)

session.commit()

# Prep line chart data 
graph of citations given per day of month

# Remove all citations with value 'NO'

In [33]:
line = reduced_df[reduced_df.citationIssued == 'YES']
line.head()

,OBJECTID,responseDate,citationIssued,lat,lon,gender,neighborhood,responseDow,responseDay,responseMonth,responseMonthName,responseYear
32602,36603,2017-07-06T13:27:32.000Z,YES,44.966620,NaN,Male,Cedar Riverside,4,06,07,Jul,2017
32614,36615,2017-07-06T14:34:37.000Z,YES,45.009403,NaN,Female,Windom Park,4,06,07,Jul,2017
32619,36620,2017-07-06T15:09:29.000Z,YES,45.011630,NaN,Male,Holland,4,06,07,Jul,2017
32625,36626,2017-07-06T15:58:31.000Z,YES,44.962690,NaN,Male,Steven's Square - Loring Heights,4,06,07,Jul,2017
32635,36636,2017-07-06T17:20:22.000Z,YES,44.999148,NaN,Male,Near - North,4,06,07,Jul,2017


# Keep only relevant columns

In [34]:
lines_df = line.drop(['OBJECTID', 'responseDate', 'responseDow','lat', 'lon', 'gender'], axis=1)
lines_df.head()

,citationIssued,neighborhood,responseDay,responseMonth,responseMonthName,responseYear
32602,YES,Cedar Riverside,06,07,Jul,2017
32614,YES,Windom Park,06,07,Jul,2017
32619,YES,Holland,06,07,Jul,2017
32625,YES,Steven's Square - Loring Heights,06,07,Jul,2017
32635,YES,Near - North,06,07,Jul,2017


# Add citation count column for graphing

In [35]:
lines_df['citationCnt'] = lines_df.groupby('neighborhood')['citationIssued'].transform('count')
lines_df.head()

,citationIssued,neighborhood,responseDay,responseMonth,responseMonthName,responseYear,citationCnt
32602,YES,Cedar Riverside,06,07,Jul,2017,109
32614,YES,Windom Park,06,07,Jul,2017,114
32619,YES,Holland,06,07,Jul,2017,235
32625,YES,Steven's Square - Loring Heights,06,07,Jul,2017,131
32635,YES,Near - North,06,07,Jul,2017,465


# Write out clean csv file

In [36]:
## Clean csv file to be saved in GitHub repository
lines_df.to_csv("static/data/clean_citation_data.csv", index=False,header=True)

# Add line data to database

In [37]:
lines_df.to_sql(citationData.__tablename__, engine, if_exists='replace',chunksize=50)

session.commit()

In [38]:
#test for data
citation = session.query(citationData.neighborhood, citationData.citationCnt).filter_by(neighborhood="Como").first()

citation

('Como', 191)